# Note

This file is only for generating 100 random pairs of locations for testing the performance of the route system. 
It is of ***no use*** for users.

In [1]:
import numpy as np
import pyproj
import networkx as nx
import random
import csv
import sys

sys.path.append('../')
from utils.Road_Network import RoadNetwork3D
from utils.City import City
from utils.Generator_Tool import generate_random_CityWeight

In [2]:
road_path = '../data/accra_road.json' # The path of 2D road data
ele_path = '../data/elevation/n05_w001_1arc_v3.tif' # The path of elevation data

accra_road = RoadNetwork3D(road_path, ele_path)
accra_road.integrate()
accra_road.create_network()

Creating network: 100%|████████████| 142611/142611 [00:01<00:00, 71562.72item/s]


In [3]:
random.seed(42)
csv_filename = 'random_locations.csv'
location_number = 100

coord_list = [['lat1', 'lon1', 'lat2', 'lon2', 'distance', 'time']]

while location_number>0:
    lat1 = random.uniform(5.55455, 5.6665)
    lon1 = random.uniform(-0.26056, -0.1696)
    city1 = City(lat1, lon1)
    
    lat2 = random.uniform(5.55455, 5.6665)
    lon2 = random.uniform(-0.26056, -0.1696)
    city2 = City(lat2, lon2)
    
    if nx.has_path(accra_road.network, accra_road.get_closest_point(city1)[0], accra_road.get_closest_point(city2)[0]):
        dist = accra_road.get_shortest_path_length(city1, city2, weight='distance')
        time = accra_road.get_shortest_path_length(city1, city2, weight='time')
        coord_list.append([lat1, lon1, lat2, lon2, dist, time])
    else: continue
    
    location_number -= 1

with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    for row in coord_list:
        writer.writerow(row)

In [4]:
#This csv file is based on 'random_locations.csv', only two columns of data from Google Maps are added
csv_file_withGoogle = 'random_locations_withGoogle.csv' 
mse_dist = 0
mean_dist = 0
mse_time = 0
mean_time = 0
count = 0
with open(csv_file_withGoogle, mode='r') as file:
    reader = csv.reader(file)

    headers = next(reader, None)
    
    if headers:
        print("CSV file contains following headers：")
        print(headers)

    for row in reader:
        mse_dist += (float(row[4])-float(row[5])*1000)**2
        mse_time += (float(row[6])-float(row[7])*60)**2
        mean_dist += float(row[5])*1000
        mean_time += float(row[7])*60
        count+=1
mse_dist = np.sqrt(mse_dist / count)
mse_time = np.sqrt(mse_time / count)
mean_dist = mean_dist/count
mean_time = mean_time/count

print('mse of distance: ', mse_dist)
print('mse percentage of distance: ', mse_dist / mean_dist)
print('mse of time: ', mse_time)
print('mse percentage of time: ', mse_time / mean_time)

CSV file contains following headers：
['lat1', 'lon1', 'lat2', 'lon2', 'distance', 'distance(Google)', 'time', 'time(Google)']
mse of distance:  920.0292186377036
mse percentage of distance:  0.11946104247714129
mse of time:  227.90850745961845
mse percentage of time:  0.23725641001417702
